In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


# Base URL of the website
base_url = 'http://books.toscrape.com/'
header = {'Accept': 'application/html'}

# List to store all book details
title = []
genre = []
rating = []
price = []
UPC = []
all_books = {}

# Iterate through all pages
page_num = 1
while True:
    page_url = base_url + 'catalogue/page-{}.html'.format(page_num)
    response = requests.get(page_url, headers=header)
    
    if response.status_code != 200:
        break
    
    page_soup = BeautifulSoup(response.content, 'html.parser')
    
    # Get all book URLs from the current page
    book_urls = [base_url + 'catalogue/' + a['href'].replace('../../../', '') for a in page_soup.select('h3 > a')]
    
    # Extract details for each book and add to the list
    for book_url in book_urls:
        book_response = requests.get(book_url)
        book_soup = BeautifulSoup(book_response.content, 'html.parser')
        
        title.append(book_soup.find('h1').text)
        genre.append(book_soup.find('ul', class_='breadcrumb').find_all('a')[2].text)
        rating.append(book_soup.find('p', class_='star-rating')['class'][1])
        price.append(book_soup.find('p', class_='price_color').text)
        UPC.append(book_soup.find('th', string='UPC').find_next_sibling('td').text)
    
    # Increment page number
    page_num += 1

# Create a dictionary to store all book details
all_books = {
    'Title': title,
    'Genre': genre,
    'Rating': rating,
    'Price': price,
    'UPC': UPC
}

# Create a DataFrame to print all the books scrape and save to CSV
try:
    if all_books:
            books_df = pd.DataFrame(all_books)
            print(f'Books Retrieved are:\n{books_df}')
            
            books_df.to_csv('books.csv', index=False)
            print(f"Scraped {len(title)} books and saved to 'books.csv'")
    else:
         print('All_books is empty')

except Exception as error:
     print(f'Error message:{error}')